In [4]:
import numpy as np

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [6]:
def sigmoid_derivative(output):
    return output * (1 - output)

In [7]:
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-9  # Avoid log(0)
    return -np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))

In [8]:
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

In [9]:
# Labels for AND gate
y = np.array([[0], [0], [0], [1]])

# Initialize weights and bias
np.random.seed(42)
weights = np.random.randn(2, 1)
bias = np.zeros((1,))

# Training parameters
learning_rate = 0.1
epochs = 10000

In [10]:
for epoch in range(epochs):
    # Forward pass
    z = np.dot(X, weights) + bias
    predictions = sigmoid(z)

    # Compute loss
    loss = binary_cross_entropy(y, predictions)

    # Backward pass (gradient computation)
    dloss_dpred = predictions - y
    dpred_dz = sigmoid_derivative(predictions)

    dz = dloss_dpred * dpred_dz

    dW = np.dot(X.T, dz) / X.shape[0]
    db = np.mean(dz)

    # Update weights and bias
    weights -= learning_rate * dW
    bias -= learning_rate * db

    # Print loss every 1000 epochs
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

Epoch 0, Loss: 0.7054
Epoch 1000, Loss: 0.3445
Epoch 2000, Loss: 0.2552
Epoch 3000, Loss: 0.2076
Epoch 4000, Loss: 0.1772
Epoch 5000, Loss: 0.1560
Epoch 6000, Loss: 0.1402
Epoch 7000, Loss: 0.1279
Epoch 8000, Loss: 0.1182
Epoch 9000, Loss: 0.1101


In [11]:
final_preds = sigmoid(np.dot(X, weights) + bias)
print("\nFinal predictions:")
print(np.round(final_preds, 2))


Final predictions:
[[0.  ]
 [0.12]
 [0.12]
 [0.86]]
